In [13]:
from datetime import date
from random import randint

from bokeh.io import output_file, show , output_notebook
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Dropdown, CheckboxGroup
from bokeh.layouts import column, gridplot, row

#output_file("data_table.html")
output_notebook()

data = dict(
        dates=[date(2014, 3, i+1) for i in range(10)],
        downloads=[randint(0, 100) for i in range(10)],
    )
source = ColumnDataSource(data)

columns = [
        TableColumn(field="dates", title="Date", formatter=DateFormatter()),
        TableColumn(field="downloads", title="Downloads"),
    ]
data_table = DataTable(source=source, columns=columns, width=400, height=280)

#show(data_table , notebook_handle=True)
#show(data_table)

menu = [("0-18", "item_1"), ("18-65", "item_2"), ("65-", "item_3")]
dropdown = Dropdown(label="Age", button_type="warning", menu=menu)
#show(dropdown, notebook_handle=True)

menu = [("Men", "item_1"), ("Women", "item_2")]
dropdown2 = Dropdown(label="Gender", button_type="warning", menu=menu)
#show(dropdown2, notebook_handle=True)

checkbox_group = CheckboxGroup(
        labels=["Disabled person"], active=[0, 1])
#show(checkbox_group, notebook_handle=True)


#l = layout([
#  [[]],
#  [sliders, plot],
#  [p1, p2, p3],
#], sizing_mode='stretch_both')
col=row(column(dropdown, dropdown2,checkbox_group), data_table)
#l=gridplot(data_table,col)
show(col)


Loading BokehJS ...